In [9]:
with open("Data.txt","r",encoding="utf-8") as f:
    raw_text=f.read()

print("total number of chars:",len(raw_text))
print(raw_text[:99])

total number of chars: 20482
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [20]:
import re

text = "Hello, how are you. This is a test."
result= re.split("(\s)",text)
print(result)

['Hello,', ' ', 'how', ' ', 'are', ' ', 'you.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [21]:
result= re.split("([,.]|\s)",text)
print(result)


['Hello', ',', '', ' ', 'how', ' ', 'are', ' ', 'you', '.', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [22]:
result=[item for item in result if item.strip()]
print(result)

['Hello', ',', 'how', 'are', 'you', '.', 'This', 'is', 'a', 'test', '.']


In [23]:
text= "Hello how are you? This-- isa test!"
result= re.split(r'([,.:;?_!"()\']|--|\s)',text)
result= [item.strip() for item in result if item.strip()]
print(result)


['Hello', 'how', 'are', 'you', '?', 'This', '--', 'isa', 'test', '!']


# Basic Tokenizer

In [26]:
preprocessed = raw_text
preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)',preprocessed)
preprocessed= [item.strip() for item in result if item.strip()]
print(preprocessed[:10])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius']


In [27]:
print(len(preprocessed))

4690


# Creating token id's

In [28]:
all_words = sorted(set(preprocessed))
vocab_size= len(all_words)
print(vocab_size)

1130


In [29]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [33]:
for i,item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break



('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [58]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [64]:
tokenizer = SimpleTokenizerV1(vocab)

text=""""It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

ids =tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [65]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [66]:
# ADDING EOS AND UNKNOWN TOKEN VALUES TO ORIGINAL VOCAB AS SPECIAL CONTEXT TOKENS

all_tokens= sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab={token:integer for integer,token in enumerate(all_tokens)}

In [68]:
len(vocab.items())

1132

In [72]:
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [76]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)                        
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed=[item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [78]:
tokenizer=SimpleTokenizerV2(vocab)
text1="Hello , do you like tea?"
text2= "In sunlit terraces of the palace."
text="<|endoftext|>".join((text1,text2))
print(text)

Hello , do you like tea?<|endoftext|>In sunlit terraces of the palace.


In [79]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1131, 956, 984, 722, 988, 1131, 7]

In [80]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|unk|> sunlit terraces of the <|unk|>.'